In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file, datas
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'volume_gal'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
field1='Capacity (Volume) [Nom]'
fields = [field1]

In [3]:
df, custom_field_df=parameters(customer_id, formatted_attribute, field1)
# [*custom_field_df]
df_cur, df_custom=datas(df,custom_field_df, field1)

257
257


In [6]:
del df_cur['bucket_id']
del df_cur['product_id']
del df_cur['image_url']
del df_cur['value']

# Custom Fields

In [7]:
df_custom=custom_field(df_custom,field1,formatted_attribute)
pat=r'''(?i)(gal)|()'''
df_custom['match'] = df_custom[curation_col].apply(lambda x: re_extract(pat, x))
df_custom[curation_col]=df_custom[curation_col].apply(lambda x: re.sub(r'(?i)(?<=oz)\.','',str(x)))
df_custom=df_custom[df_custom['match'].astype(str)!='[]']

matchcust=df_custom[['external_id',curation_col]]
print(len(matchcust))
rounding(df_custom,curation_col, 'a-np-y')

5


buckets  bucket_id  product_id external_id  \
10         Safety Wash Stations       7281   434482095  5637155758   
233  Paint, Stains, & Varnishes       6778   434495004  5637467466   
234  Paint, Stains, & Varnishes       6778   434495005  5637467467   
235  Paint, Stains, & Varnishes       6778   434495006  5637467468   
255      Sand, Mortar, & Cement       7217   434495007  5637467469   

                                                                     product_name  \
10                  Honeywell Fendall Porta Stream Self-Contained Eyewash Station   
233               Rust-Oleum Bulls Eye 1-2-3 Water-Base Primer, White, 1 gal Pail   
234                             Rust-Oleum Mold Killing Primer, White, 1 gal Pail   
235                             Rust-Oleum Odor Killing Primer, Clear, 1 gal Pail   
255  Rust-Oleum InstaPatch Ultra-Rapid Curing Concrete Repair, Gray, 1 gal Bottle   

                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
10   Fendall Porta Stream Self-Contained Eyewash Station•Pull the straps and the reservoir releases crisscross streams of Eyesaline Concentrate Solution.•Simple strap design leaves hands free to hold eyelids open while flushing.•Full-flow nozzles cover a wide area around the eyes.•16 gallon (60.5L) capacity permits thorough flushing by 0.4 gpm over a 15-minute flushing period required by current ANSI Z358.1-2014 standard.   
233                                                                                                                                                                                                             Bulls Eye 1-2-3 Water-Base Primer, White, 1 gal Pail•Water-base primer-sealer•For interior and exterior use•Sticks to all surfaces without sanding•Seals stains, graffiti and tannin bleed•Mold and mildew resistant film   
234                                                                                                                                                                                                                              Mold Killing Primer, White, 1 gal Pail•Paint directly over mold•Kills existing mold, mildew, and odor causing bacteria•Binds chalky siding, masonry, and metal•Water-based•For interior and exterior use   
235                                                                                                                                                                                                                                      Odor Killing Primer, Clear, 1 gal Pail•Kills existing odors from pets, urine, tobacco and fire•Applies white, dries clear•Compatible with any clear finish or paint•Dries in 45 minutes•Low odor   
255                                                                                                                                                                      InstaPatch Ultra-Rapid Curing Concrete Repair, Gray, 1 gal Bottle•Versatile formula can be poured or trowel applied•Color changes so you know when it's fully cured•Virtually no odor•Cures in 15 minutes or less•Can be applied in temperatures as low as -5� F   

                                                               url  \
10   https://wbmason.com/ProductDetail.aspx?ItemID=FND320002000000   
233          https://wbmason.com/ProductDetail.aspx?ItemID=RST2001   
234        https://wbmason.com/ProductDetail.aspx?ItemID=RST276049   
235        https://wbmason.com/ProductDetail.aspx?ItemID=RST305928   
255        https://wbmason.com/ProductDetail.aspx?ItemID=RST276981   

                                                 image_url      value  \
10   http://www.oppictu

# N/A Values

In [11]:
df_na=df_cur
regex_pattern_na=r'''(?i)((?<!\d)(?<!\w)(?<!\/)(?<!-)\.?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\s?gal)|(\bgal\b)|((?<!Use\s\d\-)gal)|()'''
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

df_na=df_na[(df_na['na_matches'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
df_na.head(5)

135


buckets external_id  \
0  Safety Wash Stations  5637298410   
2  Safety Wash Stations  5637297952   
3  Safety Wash Stations  5637296584   
4  Safety Wash Stations  5637298581   
7  Safety Wash Stations  5637344343   

                                                                         product_name  \
0              Sperian Saline Eye Wash Wall Station, 32 oz. Bottle, 2 Bottles/Station   
2                            Honeywell Flash Flood 3-Minute Emergency Eyewash Station   
3  Pac-Kit Twin Bottle Eye Flush Station w/Two 16oz Bottles, 3.75"D x 13.5"H x 16.5"W   
4             Honeywell Saline Eye Wash Wall Station, 16 oz. Bottle, 1 Bottle/Station   
7                                               Guardian Safety Station with Eye Wash   

                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                  Saline Eye Wash Wall Station, 32 oz. Bottle, 2 Bottles/Station•Provides immediate flushing in emergency situations•Highly visible•Bilingual instructions•Overhang prevents bottles from dust and debris   
2  Flash Flood 3-Minute Emergency Eyewash Station•Flash Flood delivers Eyesaline Solution.•Dual nozzles flush both eyes simultaneously.•When the cartridge is inserted in the base unit, the cartridge seal is broken and the Flash Flood is ready to deliver eyewash solution for three minutes.•Cartridge replacement is quick and easy.   
3                                                                                                                                                      Twin Bottle Eye Flush Station w/Two 16oz Bottles, 3.75"D x 13.5"H x 16.5"W•Sterile, isotonic buffered solution.•Easy access to bottles.•Wall-mountable.•Vital for workplace safety.   
4                                                                                                                                   Saline Eye Wash Wall Station, 16 oz. Bottle, 1 Bottle/Station•Provides immediate flushing in emergency situations•Highly visible•Bilingual instructions•Overhang prevents bottles from dust and debris   
7                                                                                                   Safety Station with Eye Wash•Convenient shower and eyewash combination.•Shower valve with stay-open chrome-plated-brass ball valve and Teflon seals.•Stainless steel actuating arm and pull rod.•Eyewash with two GS-Plus spray heads.   

                                                             url  \
0  https://wbmason.com/ProductDetail.aspx?ItemID=FND320004620000   
2  https://wbmason.com/ProductDetail.aspx?ItemID=FND320004000000   
3         https://wbmason.com/ProductDetail.aspx?ItemID=PKT24102   
4  https://wbmason.com/ProductDetail.aspx?ItemID=FND320004600000   
7        https://wbmason.com/ProductDetail.aspx?ItemID=GUAG1902P   

  Capacity (Volume) [Nom] na_matches Q:volume_gal  
0                     NaN         []          n/a  
2                     NaN         []          n/a  
3                     NaN         []          n/a  
4                     NaN         []          n/a  
7                     NaN         []          n/a

# Unit Values

In [12]:
df_oz=df_cur[df_cur['na_matches'].astype(str)!='[]']
df_oz['na_matches']=df_oz['na_matches'].apply(lambda x: sorted(x)).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r'\,','","',str(x))).apply(lambda x: re.sub(r"-",' ',str(x))).apply(lambda x: re.sub(r'(?<!\s)gal',' gal',str(x))).apply(lambda x: re.sub(r'(?<!\d)\.','0.',str(x)))
curate(df_oz, 'na_matches', formatted_attribute, 'b-fh-km-z')

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, Capacity (Volume) [Nom], na_matches, Q:volume_gal, rounding]
Index: []

In [14]:
matchoz=df_custom[['external_id',curation_col]]
print(len(matchoz))
rounding(matchoz,curation_col, 'b-fh-km-z')

5


Empty DataFrame
Columns: [external_id, Q:volume_gal, rounding]
Index: []

In [17]:
matchoz

external_id Q:volume_gal rounding
10   5637155758       16 gal       []
233  5637467466        1 gal       []
234  5637467467        1 gal       []
235  5637467468        1 gal       []
255  5637467469        1 gal       []

In [19]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matchoz))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

matchcust
matchoz
matchesna


In [ ]:
stop

# send to the folder for upload

In [20]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchoz)

In [21]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchcust)

In [22]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesna)